In [1]:
import tensorflow as tf
import keras
from keras.applications.vgg16 import VGG16
# from keras import backend as K
from keras.models import Model
from os import listdir
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from pickle import dump
import pandas as pd
import numpy as np 
from pickle import load
# from tensorflow.keras.utils import plot_model
from PIL import Image
from tqdm import tqdm 
# import matplotlib.pyplot as plt


Using TensorFlow backend.


In [2]:
gpu = tf.config.experimental.list_physical_devices('GPU')
tf.config.list_physical_devices('GPU')
tf.keras.backend.backend()

'tensorflow'

In [3]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config)

# config = tf.compat.v1.ConfigProto(allow_soft_placement=True)
# config.gpu_options.allocator_type = 'BFC'
# config.gpu_options.per_process_gpu_memory_fraction = 0.0

In [4]:
def extract_features(directory):
    # load the model
    model = VGG16()
    model.layers.pop()
    model = Model(inputs=model.inputs, outputs=model.layers[-1].output)
#     plot_model(model,to_file='model.png',show_shapes=True,show_layer_names=True)
#     print(model.summary())
    
    features = dict()
    for file in tqdm(listdir(directory)):
        img_path = directory + "/" + file
        image = load_img(img_path, target_size=(224, 224)) #size is 224,224 by default
        image = img_to_array(image)
        # Reshape data for the model
        image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
        # Prepare the image for the CNN Model model
        image = preprocess_input(image)
        # Pass image into model to get encoded features
        feature = model.predict(image, verbose=0)

        name_id = file.split('.')[0] #take the file name and use as id in dict

        features[name_id] = feature
    return features
directory = "C:/Users/Mohit/College Project/Flickr8k/Flickr8k_Dataset/Flicker8k_Dataset"
# extract_features(directory)
features = extract_features(directory)

100%|██████████████████████████████████████████████████████████████████████████████| 8091/8091 [08:21<00:00, 16.12it/s]


In [5]:
dump(features, open('encoded_image_features.pkl', 'wb'))

In [7]:
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text


In [19]:
# for making our description clean 
# iamge_id ----- image description


def load_set (doc):
    features = list()
    for line in doc.split('\n'):
        tokens = line.split()

        if len(line) < 2:
            continue

        image_id , image_desc = tokens[0] , tokens[1:]
        image_id = image_id.split('.')[0]
        image_desc = ' '.join(image_desc)
        if image_id  not in features:
            features[image_id]  =  list()
        features[image_id].append(image_desc)
    return features



In [62]:
import string
from pickle import load
def process_description_data(description):
    table = str.maketrans('','', string.punctuation)
    for key, desc_list in description.items():
        for i in range(len(desc_list)):
            desc = desc_list[i]

            desc = desc.split()

            desc = [word.lower() for word in desc]

            desc = [w.translate(table) for w in desc]

            desc = [word for word in desc if len(word)>1]

            desc = [word for word in desc if word.isalpha()]

            desc_list[i] =  ' '.join(desc)
   
  

In [23]:
def make_vocabulary(description):
    sets = set()
    for key in description.keys():
        [sets.update(d.split()) for d in description[key]]
       
    return sets
         
# my_vocab = make_vocabulary(description)
# my_vocab
# print('Length of the Vocabulary:',len(my_vocab))

In [18]:
def save_descriptions(descriptions, filename):
    lines = list()
    for key, desc_list in descriptions.items():
        for desc in desc_list:
            lines.append(key + ' ' + desc)
    data = '\n'.join(lines)
    file = open(filename, 'w')
    file.write(data)
    file.close()
# save_descriptions(description , 'description.txt')

In [1]:
# # Create a list of all the training captions
# all_train_captions = []
# for key, val in description.items():
#     for cap in val:
#         all_train_captions.append(cap)

#
# max_word_count = 8
# word_counts = {}
# nsents = 0
# for sent in all_train_captions:
#     nsents += 1
#     for w in sent.split(' '):
#         word_counts[w] = word_counts.get(w, 0) + 1

# vocab = [w for w in word_counts if word_counts[w] >= max_word_count]

# print('preprocessed words %d ' % len(vocab))
